数据沿用上课使用的数据，没有的可以下载   地址：https://github.com/yt-one/ml-teach/tree/main/datasets

编程作业：

0. 以下作业 要把周五课上的预处理全流程带上 （ColumnTransformer），可以在熟悉了课件里的代码后，考虑自己实现
    实现要求：
  a. 大多数ML算法不期望缺失值, 数字特征中的缺失值将通过用中位数替换它们来估算，。在分类特征中，缺失值将被最常见的类别替换。
  b. 大多数ML算法只接受数字输入, 类别特征将被独热编码
  c. 计算并添加一些比率特征：bedrooms_ratio、rooms_per_house和people_per_house。希望这些能更好地与房价中位数相关联
  d. 添加集群相似性特征。可能比纬度和经度对模型更有用
  e. 长尾特征被它们的对数取代，因为大多数模型更喜欢具有大致均匀分布或高斯分布的特征。
  f. 大多数ML算法更喜欢所有特征具有大致相同的尺度, 所有数值特征都将被标准化


1. 尝试支持向量机回归器(sklearn.svm.SVR)，用这个模型来做回归。
     试试这个模型的超参数，例如kernel="linear"，kernel="rbf"，不同的kernel选择下也会有不同的超参数​。  分别用GridSearchCV和RandomizedSearchCV探索性能最优（交叉验证后的预测表现最好）的超参数

     请注意，支持向量机不能扩展到大型数据集，因此应该仅在训练集的前5000个实例上训练你的模型，并且仅使用3折交叉验证，否则会要运行很久（按小时计）。
     现在不要担心支持向量机超参数的含义，将在讲支持向量机（SVM）的时候详解


2. 去了解sklearn里 SelectFromModel的用法，尝试在数据预处理流水线中添加一个SelectFromModel转换器来仅选择最重要的属性。 并用你想尝试的回归模型去训练数据（线性回归/决策树/随机森林）


3. 随堂练习: 创建一个自定义转换器，在其fit()方法中训练k近邻回归器(sklearn.neighbors.KNeighborsRegressor)，并在其transform()方法中输出模型的预测。然后将此功能添加到预处理流水线，使用纬度和经度作为此转换器的输入。这将在模型中添加一个与最近地区的房价中位数相对应的特征。  添加特征后，用想尝试的回归模型去训练数据。 训练可以采用GridSearchCV和RandomizedSearchCV探索性能最优（交叉验证后的预测表现最好）的超参数



4. 从头开始再次实现StandardScalerClone类（对数据做标准化），然后添加对inverse_transform()方法的支持：执行scaler.inverse_transform(scaler.fit_transform(X))应该返回一个非常接近X的数组。
       然后添加对特征名称的支持：如果输入是DataFrame，则在fit()方法中设置feature_names_in_。该属性类型是NumPy数组，存列的名字。
       最后，实现get_feature_names_out()方法：这个方法应该有一个可选的input_features=None参数。如果传了这个参数，这个方法应检查其长度是否匹配n_features_in_，如果有feature_names_in_属性，则应匹配feature_names_in_的长度，然后应返回input_features。
       如果input_features为None，则该方法应返回feature_names_in_（如果有这个属性），否则返回长度为n_features_in_的   np.array(["x0"，"x1"，"x2", ...])。

In [2]:

# 题目0
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_selector
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder
from sklearn.cluster import KMeans

def column_ratio(X):
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_names_in):
    return ["ratio"]  # feature names out （输出特征的名字）

def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler())

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),  # one-to-one: 1比1映射
    StandardScaler())

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))

class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        # KMeans估计器相关参数：集群数量，随机种子，KMeans是一个随机算法，依赖随机性来定位集群
        self.kmeans_ = KMeans(self.n_clusters, n_init=10,
                              random_state=self.random_state)

        # sample_weight可指定样本的相对权重, 属于KMeans算法里的超参数，训练前指定。
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self # 永远返回self

    def transform(self, X):
        # self.kmeans_.cluster_centers_ 集群中心的位置
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)

    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1., random_state=42)

default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"),
                                     StandardScaler())


preprocessing = ColumnTransformer([
        ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
        ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
        ("people_per_house", ratio_pipeline(), ["population", "households"]),
        ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population",
                               "households", "median_income"]),
        ("geo", cluster_simil, ["latitude", "longitude"]),
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
    ],
    remainder=default_num_pipeline)  # remainder，剩下的列用什么转换器，现在就剩下housing_median_age

In [6]:
# 题目1
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from scipy.stats import expon, loguniform,randint

housing = pd.read_csv("../../datasets/housing/housing.csv")
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])
strat_train_set, strat_test_set = train_test_split(
    housing, test_size=0.2, stratify=housing["income_cat"], random_state=42)  # 分层采样 单次拆分

housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()


svr_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("svr", SVR()),
])

housing_5000 = housing.iloc[:5000]
housing_labels_5000 = housing_labels.iloc[:5000]


#  随即搜索超参数，可以给一个概率分布，下面就是例子
param_distribs = {'preprocessing__geo__n_clusters': randint(low=3, high=50),
                  'svr__kernel': ['linear', 'rbf'],
                  'svr__C': loguniform(20, 200000), # loguniform(20, 200000) 表示不是在 [20, 200000] 区间内均匀采样，而是在 log(C) 这一对数空间中均匀采样。
                  'svr__gamma': expon(scale=1),}    # 表示从以 1 为尺度参数的指数分布中采样，概率密度函数为 (e^(-x)),这个分布在采样较小值时概率更高，可以更精细地探索低 gamma 区域，但也允许偶尔采样较大的值。

svr_rnd_search = RandomizedSearchCV(
    svr_pipeline, param_distributions=param_distribs, n_iter=50, cv=3,
    scoring='neg_root_mean_squared_error', random_state=42)  # 3折交叉验证，50组超参数探索

svr_rnd_search.fit(housing_5000, housing_labels_5000)
print(svr_rnd_search.best_params_)
print(svr_rnd_search.best_score_)



#   超参数之间有依赖，下面的网格展示 不同的依赖之间怎么进行搜索
#    kernel=linear，我只用探索C参数
#    kernel=rbf，  我需要探索 gamma参数
param_grid = [
        {'svr__kernel': ['linear'],
         'svr__C': [10., 30., 100., 300., 1000.,3000., 10000., 30000.0]},

        {'svr__kernel': ['rbf'],
         'svr__C': [1.0, 3.0, 10., 30., 100., 300.,1000.0],
         'svr__gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
    ]

svr_grid_search = GridSearchCV(svr_pipeline, param_grid, cv=3, scoring='neg_root_mean_squared_error')
svr_grid_search.fit(housing_5000, housing_labels_5000)

print(svr_grid_search.best_params_)
print(svr_grid_search.best_score_)


{'preprocessing__geo__n_clusters': 30, 'svr__C': 157055.10989448498, 'svr__gamma': 0.26497040005002437, 'svr__kernel': 'rbf'}
-55179.352982304925
{'svr__C': 10000.0, 'svr__kernel': 'linear'}
-69132.8010560461


In [ ]:
# 题目2
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor

preprocessing_add_feature_selection = make_pipeline(preprocessing, SelectFromModel(estimator=RandomForestRegressor(random_state=42)))
preprocessing_add_feature_selection.fit(housing, housing_labels)

preprocessing_add_feature_selection.get_feature_names_out()


# 验证上面的，就可以把 preprocessing_add_feature_selection + 回归模型 组成一个新的流水线了，然后去训练数据


In [10]:
# 题目3
from sklearn.utils.validation import check_is_fitted
from sklearn.neighbors import KNeighborsRegressor
from sklearn.utils import check_array

class NearestLabel(BaseEstimator, TransformerMixin):
    def __init__(self):
       pass

    def fit(self, X, y):
        check_array(X)
        # k个最邻近邻居的回归器
        self.knn_ = KNeighborsRegressor(weights="distance")
        self.knn_.fit(X, y)
        return self # 永远返回self

    def transform(self, X):
        check_is_fitted(self)
        result = self.knn_.predict(X)
        if result.ndim == 1:
            result = result.reshape(-1, 1)
        return result

    def get_feature_names_out(self, names=None):
        return ["Nearest Median Housing Price"]


In [3]:


from sklearn.metrics import mean_squared_error

#  这个部分 仅仅在验证类的转换是否征程
knn_reg = NearestLabel()
geo_features = housing[["latitude", "longitude"]]
knn_reg.fit_transform(geo_features, housing_labels)



preprocessing.transformers  # (名字, 转换器, 列名列表)



# 下面的代码 就在展示如何把 自定义的转换器 加到预处理流程里
from sklearn.base import clone
#
transformers = [(name, clone(transformer), columns)
                for name, transformer, columns in preprocessing.transformers]
geo_index = [name for name, _, _ in transformers].index("geo")  # "geo"这个名字在列表内出现的索引

transformers[geo_index] = ("geo", knn_reg, ["latitude", "longitude"])
#
new_geo_preprocessing = ColumnTransformer(transformers)
#
new_geo_pipeline = Pipeline([
    ('preprocessing', new_geo_preprocessing),

    #  下面的回归模型，直接用了随机搜索的最佳超参数
    ('svr', SVR(C=svr_rnd_search.best_params_["svr__C"],
                gamma=svr_rnd_search.best_params_["svr__gamma"],
                kernel=svr_rnd_search.best_params_["svr__kernel"])),
])
#
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

new_geo_pipeline.fit(housing_5000, housing_labels_5000)  # 选完超参数，开始训练
my_predictions = new_geo_pipeline.predict(X_test)  # 在测试集上验证 训练的参数

my_rmse = mean_squared_error(y_test, my_predictions, squared=False)
print(my_rmse)
# mean_squared_error(housing_labels_5000, new_geo_pipeline.predict(housing_5000), squared=False)

[('bedrooms',
  Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                  ('functiontransformer',
                   FunctionTransformer(feature_names_out=<function ratio_name at 0x00000247DD312B00>,
                                       func=<function column_ratio at 0x00000247D97804C0>)),
                  ('standardscaler', StandardScaler())]),
  ['total_bedrooms', 'total_rooms']),
 ('rooms_per_house',
  Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                  ('functiontransformer',
                   FunctionTransformer(feature_names_out=<function ratio_name at 0x00000247DD312B00>,
                                       func=<function column_ratio at 0x00000247D97804C0>)),
                  ('standardscaler', StandardScaler())]),
  ['total_rooms', 'households']),
 ('people_per_house',
  Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                  ('functiontransformer',
                   Functio

In [11]:
# 练习题5：
# fit()方法中设置feature_names_in_。该属性类型是NumPy数组，存列的名字。
#        最后，实现get_feature_names_out()方法：这个方法应该有一个可选的input_features=None参数。如果传了这个参数，这个方法应检查其长度是否匹配n_features_in_，如果有feature_names_in_属性，则应匹配feature_names_in_的长度，然后应返回input_features。
#        如果input_features为None，则该方法应返回feature_names_in_（如果有这个属性），否则返回长度为n_features_in_的   np.array(["x0"，"x1"，"x2", ...])。
class StandardScalerClone(BaseEstimator, TransformerMixin):
    def __init__(self, with_mean=True):  # 没有 *args, **kwargs, sklearn的规范
        self.with_mean = with_mean

    def fit(self, X, y=None):  # 即使不用y，也需要它
        if isinstance(X, pd.DataFrame):
            self.feature_names_in_ = X.columns.values

        X = check_array(X)  # 检查X是不是数组
        self.mean_ = X.mean(axis=0)
        self.scale_ = X.std(axis=0)
        self.n_features_in_ = X.shape[1]  # 所有估计器会把输入特征的数量存下来
        return self  # 永远返回 self!

    def transform(self, X):
        check_is_fitted(self)  # 检查是否适配过数据 （检查是否有那些下划线结尾的参数)
        X = check_array(X)
        assert self.n_features_in_ == X.shape[1]  # 检查训练时的输入特征数量 和 要transofrm的时候的输入特征数量是不是一样的
        if self.with_mean:  # 为什么有这个参数？ 可以控制 稀疏矩阵的标准化，希望稀疏矩阵标准化后 还是稀疏的
            X = X - self.mean_
        return X / self.scale_

    def inverse_transform(self, X_transformed):
        check_is_fitted(self)
        X_transformed = check_array(X_transformed)
        assert self.n_features_in_ == X_transformed.shape[1]

        X = X_transformed * self.scale_
        if self.with_mean:
            X = X + self.mean_
        return X

    def get_feature_names_out(self, input_features=None):
        if input_features:
            assert len(input_features) == len(self.feature_names_in_) if hasattr(self, "feature_names_in_") else len(input_features) == self.n_features_in_
            return input_features
        return self.feature_names_in_ if hasattr(self, "feature_names_in_") else np.array([f"x{i}" for i in range(self.n_features_in_)])



In [13]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

housing_num = housing.select_dtypes(include=[np.number])
X = imputer.fit_transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing_num.index)

ssc = StandardScalerClone(with_mean=False)

new_housing = ssc.fit_transform(housing_tr)
np.ptp(new_housing, axis=0)
np.ptp(X, axis=0)

new_X = ssc.inverse_transform(new_housing)
np.allclose(X, new_X)

ssc.fit(housing_tr)
ssc.get_feature_names_out()

True